#Predicting total discount (after being classified as non zero discount from previous model)

Importing libraries

In [ ]:
 import pandas as pd
 import numpy as np
 from xgboost import XGBRegressor
 from sklearn.model_selection import train_test_split
 from sklearn import metrics

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot

Upload "Data_Encoded.csv" from the INPUT folder provided in the drive link

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
data_r = pd.read_csv('/content/Data_Encoded.csv')

In [ ]:
data_r.head()

,total_volume_2019,total_volume_2018,city_tier,gross_turnover_2019,on_invoice_discount,off_invoice_discount,product_volume_2019,tax,total_discount,gto-tax,Premium,Entertainment Led,Food Led,Institutional,Not applicable,Wholesaler,BULK,CAN,KEG,PERFECTDRAFT,RETURNABLE,sub_segment_Bar,sub_segment_Hybrid,sub_segment_Beer bar,sub_segment_Restaurant,sub_segment_Institutional,sub_segment_Sports Venue,brand_LEFFE,brand_JUPILER,brand_HOEGAARDEN,brand_TRIPEL KARMELIET,brand_BELLE VUE,brand_STELLA ARTOIS,sub_brand_JUPILER PILS,sub_brand_LEFFE BLONDE,sub_brand_HOEGAARDEN WHITE,"sub_brand_JUPILER 0,0",sub_brand_LEFFE BRUNE,sub_brand_TRIPEL KARMELIET,sub_brand_STELLA ARTOIS REGULAR,sub_brand_HOEGAARDEN ROSEE,province_West Flanders,province_Brussels Capital Region,province_Liège,province_Flemish Brabant,province_East Flanders,province_Hainaut,province_Antwerp,province_Limburg,province_Namur
0,0.48,0.557,1,141.20,0.00,0.0,0.48,12.23,0.00,128.97,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0.45,0.540,2,113.65,0.00,0.0,0.45,7.85,0.00,105.80,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,270.97,225.720,2,61551.06,14511.46,0.0,238.57,3577.69,14511.46,57973.37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,270.97,225.720,2,6001.00,1425.19,180.0,20.40,519.41,1605.19,5481.59,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,270.97,225.720,2,3051.36,718.32,0.0,12.00,288.63,718.32,2762.73,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Dropping redundant columns/ Getting the data ready to train

In [ ]:
data_r['2019-2018'] = data_r['total_volume_2019']-data_r['total_volume_2018']
data_r.drop(["total_volume_2018"],axis = 1, inplace = True)

Data set having total discounts greater than zero are chosen 

In [ ]:
data_r = data_r[data_r['total_discount']>0]

In [ ]:
data_r.reset_index(inplace = True)

Dropping redundant columns/ Getting the data ready to train

In [ ]:
X_tds_r = data_r.drop(["total_discount","on_invoice_discount","off_invoice_discount","gross_turnover_2019","tax"], axis=1, inplace = False) 

In [ ]:
Y_tds_r = data_r["total_discount"]


In [ ]:
X_tds_r.drop('index', axis =1 ,inplace= True)
X_tds_r.head()

,total_volume_2019,city_tier,product_volume_2019,gto-tax,Premium,Entertainment Led,Food Led,Institutional,Not applicable,Wholesaler,BULK,CAN,KEG,PERFECTDRAFT,RETURNABLE,sub_segment_Bar,sub_segment_Hybrid,sub_segment_Beer bar,sub_segment_Restaurant,sub_segment_Institutional,sub_segment_Sports Venue,brand_LEFFE,brand_JUPILER,brand_HOEGAARDEN,brand_TRIPEL KARMELIET,brand_BELLE VUE,brand_STELLA ARTOIS,sub_brand_JUPILER PILS,sub_brand_LEFFE BLONDE,sub_brand_HOEGAARDEN WHITE,"sub_brand_JUPILER 0,0",sub_brand_LEFFE BRUNE,sub_brand_TRIPEL KARMELIET,sub_brand_STELLA ARTOIS REGULAR,sub_brand_HOEGAARDEN ROSEE,province_West Flanders,province_Brussels Capital Region,province_Liège,province_Flemish Brabant,province_East Flanders,province_Hainaut,province_Antwerp,province_Limburg,province_Namur,2019-2018
0,270.97,2,238.57,57973.37,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,45.25
1,270.97,2,20.40,5481.59,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,45.25
2,270.97,2,12.00,2762.73,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,45.25
3,29.61,4,11.34,2166.28,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.75
4,29.61,4,17.37,3318.07,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.75


In [ ]:
Y_tds_r.head()

0    14511.46
1     1605.19
2      718.32
3      686.13
4     1050.98
Name: total_discount, dtype: float64

Train test dataset splitting

In [ ]:
X_tds_r_train, X_tds_r_test, y_tds_r_train, y_tds_r_test = train_test_split(X_tds_r,Y_tds_r,test_size = 0.2)

In [ ]:
X_tds_r_train.shape

(26044, 45)

Final training model after hyperparameter tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pickle

In [ ]:
rf_model = RandomForestRegressor(bootstrap= True,max_depth=15,max_features= 24,min_samples_leaf= 2, min_samples_split=3, n_estimators= 500)
rf_model.fit(X_tds_r_train,y_tds_r_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features=24, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Predicting the testdataset

In [ ]:
pred = rf_model.predict(X_tds_r_test)

Mean Absolute Error (MAE)

In [ ]:
metrics.mean_absolute_error(y_tds_r_test,pred)

224.14570128416648

Mean Squared error (MSE)

In [ ]:
metrics.mean_squared_error(y_tds_r_test,pred)

1841904.7018508022

R^2 

In [ ]:
metrics.r2_score(y_tds_r_test,pred)

0.8879354787048992

Model is good

Exporting model as pickle file( Commented )

In [ ]:
# with open('hypertuned_rf_total discount.pickle','wb') as modelFile:
#      pickle.dump(rf_model,modelFile)

Predicting the training dataset ( Commented )

In [ ]:
# pred = rf_model.predict(X_tds_r_train)

In [ ]:
# metrics.mean_absolute_error(y_tds_r_train,pred)

131.53029699167962

In [ ]:
# metrics.mean_squared_error(y_tds_r_train,pred)

666612.9699533536

In [ ]:
# metrics.r2_score(y_tds_r_train,pred)

0.957661786000332

#Hyper parameter tuning ( Commented )

Randomize seach CV

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestRegressor

In [ ]:
# n_estimators = [500]
# max_features = [22,24] #keep numerical values of 30-50% of total features
# max_depth = [15]
# min_samples_split = [3]
# min_samples_leaf = [2]
# bootstrap = [True]


# random_grid = { 'n_estimators': n_estimators, 'max_features':max_features, 'max_depth':max_depth, 'min_samples_split':min_samples_split, 'min_samples_leaf':min_samples_leaf,
#                'bootstrap': bootstrap}
# print(random_grid)

{'n_estimators': [500], 'max_features': [22, 24], 'max_depth': [15], 'min_samples_split': [3], 'min_samples_leaf': [2], 'bootstrap': [True]}


In [ ]:
# rf_new = RandomForestRegressor()
# rf_randomcv = RandomizedSearchCV(estimator= rf_new, param_distributions= random_grid, n_iter= 5, cv=3, verbose=1,random_state=69)
# rf_randomcv.fit(X_tds_r_train,y_tds_r_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [ ]:
# rf_randomcv.best_params_

{'bootstrap': True,
 'max_depth': 15,
 'max_features': 22,
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 500}

Predicting the testing dataset

In [ ]:
# random_pred = rf_randomcv.best_estimator_.predict(X_tds_r_test)

In [ ]:
# metrics.mean_absolute_error(y_tds_r_test,random_pred)

224.15282494603682

In [ ]:
# metrics.mean_squared_error(y_tds_r_test,random_pred)

1828277.93843426

In [ ]:
# metrics.r2_score(y_tds_r_test,random_pred)

0.8887645534759999

Predicting training dataset

In [ ]:
# random_pred = rf_randomcv.best_estimator_.predict(X_tds_r_train)

In [ ]:
# metrics.mean_absolute_error(y_tds_r_train,random_pred)

133.45828268477638

In [ ]:
# metrics.mean_squared_error(y_tds_r_train,random_pred)

670482.7533245874

In [ ]:
# metrics.r2_score(y_tds_r_train,random_pred)

0.95741600662926